In [56]:
# %%bash

# rm test.db
# sqlite3 test.db '.read data/RFRGdata.sql'

In [57]:
# %load soundex.py
"""
soundex module conforming to Knuth's algorithm
implementation 2000-12-24 by Gregory Jorgensen
public domain
available at:
http://code.activestate.com/recipes/52213-soundex-algorithm/
"""


def soundex(name, len=4):
    # digits holds the soundex values for the alphabet
    digits = '01230120022455012623010202'
    sndx = ''
    fc = ''

    # translate alpha chars in name to soundex digits
    for c in name.upper():
        if c.isalpha():
            if not fc:
                fc = c   # remember first letter
            d = digits[ord(c)-ord('A')]
            # duplicate consecutive soundex digits are skipped
            if not sndx or (d != sndx[-1]):
                sndx += d

    # replace first digit with first alpha character
    sndx = fc + sndx[1:]

    # remove all 0s from the soundex code
    sndx = sndx.replace('0', '')

    # return soundex code padded to len characters
    return (sndx + (len * '0'))[:len]

# Entity Matching

Approach:

1. Run SQL to find project pairs with matching URLs. Add these to the candidate list.
2. Run SQL to find project pairs with matching names. Add these to the candidate list.
3. For each candidate pair.
    1. Calculate the levenshtein distance on URLs.
    2. Calculate the levenshtein distance on names.
    3. Set Boolean: is the RubyForge name found in the RubyGems name?
    4. Set Boolean: is the RubyForge name found in the RubyGems URL?
    5. Set Boolean: is the RubyForge developer found on the list of RubyGems developer.

In [58]:
import sqlite3
from nltk.metrics import edit_distance
from pprint import pprint

In [71]:
conn = sqlite3.connect('test.db')

In [60]:
res = conn.cursor().execute('select sql from sqlite_master where name="book_rg_entity_people"').fetchall()
pprint(res[0])

('CREATE TABLE `book_rg_entity_people` (\n'
 '  `project_name` varchar(100) NOT NULL,\n'
 '  `person_name` varchar(100) NOT NULL,\n'
 '  PRIMARY KEY (`project_name`,`person_name`)\n'
 ')',)


In [72]:
res = conn.cursor().execute('select sql from sqlite_master where name="book_rf_entity_people"').fetchall()
pprint(res[0])

('CREATE TABLE `book_rf_entity_people` (\n'
 '  `project_name` varchar(100) NOT NULL,\n'
 '  `dev_username` varchar(100) NOT NULL,\n'
 '  `dev_realname` varchar(100) NOT NULL,\n'
 '  PRIMARY KEY (`project_name`,`dev_username`)\n'
 ')',)


In [73]:
res = conn.cursor().execute('select sql from sqlite_master where name="book_entity_matches"').fetchall()
pprint(res[0])

('CREATE TABLE book_entity_matches (\n'
 '        rf_project_name varchar(100) NOT NULL,\n'
 '        rg_project_name varchar(100) NOT NULL,\n'
 '        url_levenshtein int(11) DEFAULT NULL,\n'
 '        rf_name_soundex varchar(5) DEFAULT NULL,\n'
 '        rg_name_soundex varchar(5) DEFAULT NULL,\n'
 '        name_levenshtein int(11) DEFAULT NULL,\n'
 '        rf_name_in_rg_name tinyint(1) DEFAULT NULL,\n'
 '        rf_name_in_rg_url tinyint(1) DEFAULT NULL,\n'
 '        rf_dev_in_rg_dev tinyint(1) DEFAULT NULL,\n'
 '        PRIMARY KEY (rf_project_name, rg_project_name)\n'
 '    )',)


In [62]:
conn.cursor().execute('SELECT name FROM sqlite_master WHERE type="table"').fetchall()

[('book_rf_entities',),
 ('book_rf_entity_people',),
 ('book_rf_entity_topics',),
 ('book_rg_entities',),
 ('book_rg_entity_people',),
 ('book_entity_matches',)]

In [63]:
# NOTE: rf: RubyForge, rg: RubyGems.
conn.cursor().execute('''
    CREATE TABLE IF NOT EXISTS book_entity_matches (
        rf_project_name varchar(100) NOT NULL,
        rg_project_name varchar(100) NOT NULL,
        url_levenshtein int(11) DEFAULT NULL,
        rf_name_soundex varchar(5) DEFAULT NULL,
        rg_name_soundex varchar(5) DEFAULT NULL,
        name_levenshtein int(11) DEFAULT NULL,
        rf_name_in_rg_name tinyint(1) DEFAULT NULL,
        rf_name_in_rg_url tinyint(1) DEFAULT NULL,
        rf_dev_in_rg_dev tinyint(1) DEFAULT NULL,
        PRIMARY KEY (rf_project_name, rg_project_name)
    )
''').fetchone()

In [64]:
cursor = conn.cursor()

In [65]:
# Get all projects with matching URLs.
try:
    cursor.execute('''
        INSERT INTO book_entity_matches 
            (rf_project_name, rg_project_name)
        SELECT rf.project_name, rg.project_name
        FROM book_rf_entities rf
        INNER JOIN book_rg_entities rg
        ON rf.url = rg.url
    ''')
    conn.commit()
except sqlite3.IntegrityError as e:
    print('executed')

executed


In [66]:
# Get projects that have matching project names.
try:
    res = cursor.execute('''
        INSERT INTO book_entity_matches 
            (rf_project_name, rg_project_name)
        SELECT rf.project_name, rg.project_name
        FROM book_rf_entities rf
        INNER JOIN book_rg_entities rg
        ON rf.project_name = rg.project_name
        WHERE rf.project_name NOT IN (
            SELECT bem.rf_project_name
            FROM book_entity_matches bem
        )
    ''')
    conn.commit()
except sqlite3.IntegrityError as e:
    print('executed')

In [67]:
# Calculate the string metrics for each pair.
c = conn.cursor()
c.execute('''
    SELECT bem.rf_project_name,
           bem.rg_project_name,
           rfe.url,
           rge.url
    FROM book_entity_matches bem
    INNER JOIN book_rg_entities rge
        ON bem.rg_project_name = rge.project_name
    INNER JOIN book_rf_entities rfe
        ON bem.rf_project_name = rfe.project_name
    ORDER BY bem.rf_project_name
''')

project_pairs = c.fetchall()
project_pairs[0]

('aafc',
 'acts_as_flux_capacitor',
 'http://aafc.rubyforge.org',
 'http://aafc.rubyforge.org')

In [68]:
cursor = conn.cursor()

for (rf_name, rg_name, rf_url, rg_url) in project_pairs:
    rf_name_lower = rf_name.lower()
    rg_name_lower = rg_name.lower()
    rf_url_lower = rf_url.lower()
    rg_url_lower = rg_url.lower()
    
    lev_names = edit_distance(rf_name_lower, rg_name_lower)
    lev_urls = edit_distance(rf_url_lower, rg_url_lower)
    soundex_rf_name = soundex(rf_name_lower)
    soundex_rg_name = soundex(rg_name_lower)
    
    # Is the RF project name inside the RG project name.
    rf_in_rg_name = 1 if rf_name_lower in rg_name_lower else 0
    rf_in_rg_url = 1 if rf_name_lower in rg_url_lower else 0
    
    # Do RF devs match the RG devs.
    result = cursor.execute('''
        SELECT rf.dev_username, rf.dev_realname
        FROM book_rf_entity_people rf
        WHERE rf.project_name = ?
        AND rf.dev_username IN (
            SELECT rg.person_name
            FROM book_rg_entity_people rg
            WHERE rg.project_name = ?
        )
        OR rf.dev_realname IN (
            SELECT rg.person_name
            FROM book_rg_entity_people rg
            WHERE rg.project_name = ?
        )
    ''', (rf_name, rg_name, rg_name)).fetchone()
    
    rfdev_in_rgdev = 1 if result is not None else 0
    
    cursor.execute('''
        UPDATE book_entity_matches
        SET rf_name_soundex = ?,
            rg_name_soundex = ?,
            url_levenshtein = ?,
            name_levenshtein = ?,
            rf_name_in_rg_name = ?,
            rf_name_in_rg_url = ?,
            rf_dev_in_rg_dev = ?
        WHERE rf_project_name = ?
        AND rg_project_name = ?
    ''', (soundex_rf_name,
          soundex_rg_name,
          lev_urls,
          lev_names,
          rf_in_rg_name,
          rf_in_rg_url,
          rfdev_in_rgdev,
          rf_name,
          rg_name))
conn.commit()

In [78]:
cursor = conn.cursor()

count = cursor.execute('''
    SELECT count(*) FROM book_entity_matches
''').fetchone()[0]

print(f'found {count} matches')

res = cursor.execute('''
    SELECT * FROM book_entity_matches LIMIT 10
''').fetchall()

for (rf_project_name, 
     rg_project_name, 
     url_levenshtein,
     rf_name_soundex,
     rg_name_soundex,
     name_levenshtein,
     rf_name_in_rg_name,
     rf_name_in_rg_url,
     rf_dev_in_rg_dev
    ) in res:
    
    print('rf_project_name', rf_project_name)
    print('rg_project_name', rg_project_name)
    print('url_levenshtein', url_levenshtein)
    print('rf_name_soundex', rf_name_soundex)
    print('rg_name_soundex', rg_name_soundex)
    print('name_levenshtein', name_levenshtein)
    print('rf_name_in_rg_name', rf_name_in_rg_name)
    print('rf_name_in_rg_url', rf_name_in_rg_url)
    print('rf_dev_in_rg_dev', rf_dev_in_rg_dev)
    print('')

found 5791 matches
rf_project_name jabber4r
rg_project_name jabber4r
url_levenshtein 0
rf_name_soundex J160
rg_name_soundex J160
name_levenshtein 0
rf_name_in_rg_name 1
rf_name_in_rg_url 1
rf_dev_in_rg_dev 1

rf_project_name muravey-tools
rg_project_name radiant
url_levenshtein 0
rf_name_soundex M613
rg_name_soundex R353
name_levenshtein 10
rf_name_in_rg_name 0
rf_name_in_rg_url 0
rf_dev_in_rg_dev 0

rf_project_name muravey-tools
rg_project_name radiant-archive-extension
url_levenshtein 0
rf_name_soundex M613
rg_name_soundex R353
name_levenshtein 21
rf_name_in_rg_name 0
rf_name_in_rg_url 0
rf_dev_in_rg_dev 0

rf_project_name muravey-tools
rg_project_name radiant-clipped-extension
url_levenshtein 0
rf_name_soundex M613
rg_name_soundex R353
name_levenshtein 21
rf_name_in_rg_name 0
rf_name_in_rg_url 0
rf_dev_in_rg_dev 0

rf_project_name muravey-tools
rg_project_name radiant-clipped-extension-add-group
url_levenshtein 0
rf_name_soundex M613
rg_name_soundex R353
name_levenshtein 29
rf_name_

In [86]:
cusor = conn.cursor()

# Find the type 1 false positive, FP1.
# The only matching params is the URL.

fp1 = cursor.execute('''
    SELECT rf.url, bme.*
    FROM book_entity_matches bme
    INNER JOIN book_rf_entities rf
        ON (bme.rf_project_name = rf.project_name)
    WHERE url_levenshtein = 0
    AND rf_name_soundex <> rg_name_soundex
    AND name_levenshtein > 0
    AND rf_name_in_rg_name = 0
    AND rf_name_in_rg_url = 0
    AND rf_dev_in_rg_dev = 0
''').fetchall()
len(fp1)

119

In [90]:
# Find the type 2 false positive, FP2.
fp2 = cursor.execute('''
    SELECT rf.url, rg.url, bme.*
    FROM book_entity_matches bme
    INNER JOIN book_rf_entities rf
        ON (bme.rf_project_name = rf.project_name)
    INNER JOIN book_rg_entities rg
        ON (bme.rg_project_name = rg.project_name)
    WHERE name_levenshtein = 0
    AND url_levenshtein > 0
    AND rf_name_in_rg_url = 0
    AND rf_dev_in_rg_dev = 0
''').fetchall()
len(fp2)

86

In [93]:
# Find the true positives, which are the exact matches.
tp = cursor.execute('''
    SELECT rf.url, rg.url, bme.*
    FROM book_entity_matches bme
    INNER JOIN book_rf_entities rf
        ON bme.rf_project_name = rf.project_name
    INNER JOIN book_rg_entities rg
        ON bme.rg_project_name = rg.project_name
    WHERE name_levenshtein = 0
    AND url_levenshtein = 0
    AND rf_name_in_rg_url = 1
    AND rf_dev_in_rg_dev = 1
    AND rf_name_in_rg_name = 1
''').fetchall()
len(tp)

1091

In [94]:
conn.close()